In [25]:
# Importing Dependencies
import pandas as pd
import requests
import time
from datetime import datetime, timedelta

#Import API Key
from api_key import wx_api_key

### Creating List of Dates

In [26]:
# Creating List of Dates
dates = []

# Assuming 365 Days in a Year
number_of_days = 365

# Inputting Start Date
start_date_obj = datetime(2015, 1, 1)

# For Loop to Create List of Dates
for day in range(number_of_days):
    new_date = start_date_obj + timedelta(day)
    new_date = new_date.strftime("%Y-%m-%d")
    dates.append(new_date)

### Creating DataFrame of New York Airports

In [51]:
#read the csv files in
airport = "../Resources/airports.csv"

#create df for airports
airport_df = pd.read_csv(airport)

# Locating Airports in the US that have Small Airports
sm_arprts = airport_df.loc[(airport_df['iso_country']=='US') & (airport_df['type']=='small_airport') & (airport_df['iso_region']=='US-NY')]
                                                                
# Dropping NaN in IATA                                                              
sm_arprts = sm_arprts.dropna(subset=['iata_code'])

# Locating Airports in the US that have Medium Airports
md_arprts = airport_df.loc[(airport_df['iso_country']=='US') & (airport_df['type']=='medium_airport') & (airport_df['iso_region']=='US-NY')]
                                                                
# Dropping NaN in IATA                                                              
md_arprts = md_arprts.dropna(subset=['iata_code'])

# Locating Airports in the US that have Medium Airports
lg_arprts = airport_df.loc[(airport_df['iso_country']=='US') & (airport_df['type']=='large_airport') & (airport_df['iso_region']=='US-NY')]
                                                                
# Dropping NaN in IATA                                                              
lg_arprts = lg_arprts.dropna(subset=['iata_code'])

# Concatinating DataFrames into One
arprts = pd.concat([sm_arprts, md_arprts, lg_arprts])

### Creating List of IATA Codes

In [52]:
iata_codes = arprts['iata_code']

# List of Major Airports in New York
locs = []

# Looping Through Iata Codes
for code in iata_codes:
    locs.append(code)

###  Perform API Calls

In [53]:
# Base URL
base_url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx?"

# WorldWeather Online Has Two Formats with XML as the Default
format_output = 'json'

# Time Interval, Set to 6 Hourly Intervals
tp = 6

# Include location
incLoc = 'yes'

# Building Query URL
query_url = f"{base_url}key={wx_api_key}&format={format_output}&tp={tp}&includelocation={incLoc}"

In [54]:
# Creating Empty Lists
date_list = []
time_list = []
loc_list = []
lat =[]
lng = []
tempF = []
DewPointF = []
windspeedMiles = []
WindGustMiles = []
winddir16Point = []
weatherDesc = []
precipInches = []
humidity = []
visibilityMiles = []
pressure = []
cloudcover = []
HeatIndexF = []
WindChillF = []
totalSnow_cm = []


record_count = 1

# Using a For Loop to Make Successive API Calls
print('Beginning Data Retrieval')
print('------------------------')

# Loop Through Each Time Interval (*Hard Coded for 6 Hourly Data*)
for time in range(4):
    # Loop Through Dates        
    for date in dates:
        # Loop Through IATA Codes
        for loc in locs:
            # Making Requests
            if record_count >= 50:
                time.sleep(60)
                record_count = 0
                response = requests.get(f'{query_url}&date={date}&q={loc}').json()
                record_count += 1
            
            # Using Try/Except to Grab Variables if Available
            try:
                print(f'Processing Time Interval {time} for {date} for {loc}')
                loc_list.append(loc)
                date_list.append(date)
                time_list.append(response['data']['weather'][0]['hourly'][time]['time'])
                lat.append(response['data']['nearest_area'][0]['latitude'])
                lng.append(response['data']['nearest_area'][0]['longitude'])
                totalSnow_cm.append(response['data']['weather'][0]['totalSnow_cm'])
                tempF.append(response['data']['weather'][0]['hourly'][time]['tempF'])
                DewPointF.append(response['data']['weather'][0]['hourly'][time]['DewPointF'])
                windspeedMiles.append(response['data']['weather'][0]['hourly'][time]['windspeedMiles'])
                WindGustMiles.append(response['data']['weather'][0]['hourly'][time]['WindGustMiles'])
                winddir16Point.append(response['data']['weather'][0]['hourly'][time]['winddir16Point'])
                weatherDesc.append(response['data']['weather'][0]['hourly'][time]['weatherDesc'][0]['value'])
                precipInches.append(response['data']['weather'][0]['hourly'][time]['precipInches'])
                humidity.append(response['data']['weather'][0]['hourly'][time]['humidity'])
                visibilityMiles.append(response['data']['weather'][0]['hourly'][time]['visibilityMiles'])
                pressure.append(response['data']['weather'][0]['hourly'][time]['pressure'])
                cloudcover.append(response['data']['weather'][0]['hourly'][time]['cloudcover'])
                HeatIndexF.append(response['data']['weather'][0]['hourly'][time]['HeatIndexF'])
                WindChillF.append(response['data']['weather'][0]['hourly'][time]['WindChillF'])
            except KeyError:
                print('Day not found. Skipping...')
print('------------------------')
print('Data Retrieval Complete')
print('------------------------')

Beginning Data Retrieval
------------------------
Processing Time Interval 0 for 2015-01-01 for AXB
Processing Time Interval 0 for 2015-01-01 for IUA
Processing Time Interval 0 for 2015-01-01 for FID
Processing Time Interval 0 for 2015-01-01 for HCC
Processing Time Interval 0 for 2015-01-01 for COP
Processing Time Interval 0 for 2015-01-01 for CTO
Processing Time Interval 0 for 2015-01-01 for DKK
Processing Time Interval 0 for 2015-01-01 for DSV
Processing Time Interval 0 for 2015-01-01 for ELZ
Processing Time Interval 0 for 2015-01-01 for FOK
Processing Time Interval 0 for 2015-01-01 for HTO
Processing Time Interval 0 for 2015-01-01 for WSH
Processing Time Interval 0 for 2015-01-01 for LKP
Processing Time Interval 0 for 2015-01-01 for MGJ
Processing Time Interval 0 for 2015-01-01 for MSV
Processing Time Interval 0 for 2015-01-01 for MTP
Processing Time Interval 0 for 2015-01-01 for CTX
Processing Time Interval 0 for 2015-01-01 for SXY
Processing Time Interval 0 for 2015-01-01 for OGS


Processing Time Interval 0 for 2015-01-10 for JHW
Processing Time Interval 0 for 2015-01-10 for MSS
Processing Time Interval 0 for 2015-01-10 for PBG
Processing Time Interval 0 for 2015-01-10 for POU
Processing Time Interval 0 for 2015-01-10 for RME
Processing Time Interval 0 for 2015-01-10 for SCH
Processing Time Interval 0 for 2015-01-10 for SLK
Processing Time Interval 0 for 2015-01-10 for SWF
Processing Time Interval 0 for 2015-01-10 for BUF
Processing Time Interval 0 for 2015-01-10 for JFK
Processing Time Interval 0 for 2015-01-10 for LGA
Processing Time Interval 0 for 2015-01-10 for ROC
Processing Time Interval 0 for 2015-01-10 for SYR
Processing Time Interval 0 for 2015-01-11 for AXB
Processing Time Interval 0 for 2015-01-11 for IUA
Processing Time Interval 0 for 2015-01-11 for FID
Processing Time Interval 0 for 2015-01-11 for HCC
Processing Time Interval 0 for 2015-01-11 for COP
Processing Time Interval 0 for 2015-01-11 for CTO
Processing Time Interval 0 for 2015-01-11 for DKK


Processing Time Interval 0 for 2015-01-26 for JHW
Processing Time Interval 0 for 2015-01-26 for MSS
Processing Time Interval 0 for 2015-01-26 for PBG
Processing Time Interval 0 for 2015-01-26 for POU
Processing Time Interval 0 for 2015-01-26 for RME
Processing Time Interval 0 for 2015-01-26 for SCH
Processing Time Interval 0 for 2015-01-26 for SLK
Processing Time Interval 0 for 2015-01-26 for SWF
Processing Time Interval 0 for 2015-01-26 for BUF
Processing Time Interval 0 for 2015-01-26 for JFK
Processing Time Interval 0 for 2015-01-26 for LGA
Processing Time Interval 0 for 2015-01-26 for ROC
Processing Time Interval 0 for 2015-01-26 for SYR
Processing Time Interval 0 for 2015-01-27 for AXB
Processing Time Interval 0 for 2015-01-27 for IUA
Processing Time Interval 0 for 2015-01-27 for FID
Processing Time Interval 0 for 2015-01-27 for HCC
Processing Time Interval 0 for 2015-01-27 for COP
Processing Time Interval 0 for 2015-01-27 for CTO
Processing Time Interval 0 for 2015-01-27 for DKK


Processing Time Interval 0 for 2015-02-05 for SLK
Processing Time Interval 0 for 2015-02-05 for SWF
Processing Time Interval 0 for 2015-02-05 for BUF
Processing Time Interval 0 for 2015-02-05 for JFK
Processing Time Interval 0 for 2015-02-05 for LGA
Processing Time Interval 0 for 2015-02-05 for ROC
Processing Time Interval 0 for 2015-02-05 for SYR
Processing Time Interval 0 for 2015-02-06 for AXB
Processing Time Interval 0 for 2015-02-06 for IUA
Processing Time Interval 0 for 2015-02-06 for FID
Processing Time Interval 0 for 2015-02-06 for HCC
Processing Time Interval 0 for 2015-02-06 for COP
Processing Time Interval 0 for 2015-02-06 for CTO
Processing Time Interval 0 for 2015-02-06 for DKK
Processing Time Interval 0 for 2015-02-06 for DSV
Processing Time Interval 0 for 2015-02-06 for ELZ
Processing Time Interval 0 for 2015-02-06 for FOK
Processing Time Interval 0 for 2015-02-06 for HTO
Processing Time Interval 0 for 2015-02-06 for WSH
Processing Time Interval 0 for 2015-02-06 for LKP


Processing Time Interval 0 for 2015-02-24 for LGA
Processing Time Interval 0 for 2015-02-24 for ROC
Processing Time Interval 0 for 2015-02-24 for SYR
Processing Time Interval 0 for 2015-02-25 for AXB
Processing Time Interval 0 for 2015-02-25 for IUA
Processing Time Interval 0 for 2015-02-25 for FID
Processing Time Interval 0 for 2015-02-25 for HCC
Processing Time Interval 0 for 2015-02-25 for COP
Processing Time Interval 0 for 2015-02-25 for CTO
Processing Time Interval 0 for 2015-02-25 for DKK
Processing Time Interval 0 for 2015-02-25 for DSV
Processing Time Interval 0 for 2015-02-25 for ELZ
Processing Time Interval 0 for 2015-02-25 for FOK
Processing Time Interval 0 for 2015-02-25 for HTO
Processing Time Interval 0 for 2015-02-25 for WSH
Processing Time Interval 0 for 2015-02-25 for LKP
Processing Time Interval 0 for 2015-02-25 for MGJ
Processing Time Interval 0 for 2015-02-25 for MSV
Processing Time Interval 0 for 2015-02-25 for MTP
Processing Time Interval 0 for 2015-02-25 for CTX


...

Processing Time Interval 3 for 2015-10-25 for PBG
Processing Time Interval 3 for 2015-10-25 for POU
Processing Time Interval 3 for 2015-10-25 for RME
Processing Time Interval 3 for 2015-10-25 for SCH
Processing Time Interval 3 for 2015-10-25 for SLK
Processing Time Interval 3 for 2015-10-25 for SWF
Processing Time Interval 3 for 2015-10-25 for BUF
Processing Time Interval 3 for 2015-10-25 for JFK
Processing Time Interval 3 for 2015-10-25 for LGA
Processing Time Interval 3 for 2015-10-25 for ROC
Processing Time Interval 3 for 2015-10-25 for SYR
Processing Time Interval 3 for 2015-10-26 for AXB
Processing Time Interval 3 for 2015-10-26 for IUA
Processing Time Interval 3 for 2015-10-26 for FID
Processing Time Interval 3 for 2015-10-26 for HCC
Processing Time Interval 3 for 2015-10-26 for COP
Processing Time Interval 3 for 2015-10-26 for CTO
Processing Time Interval 3 for 2015-10-26 for DKK
Processing Time Interval 3 for 2015-10-26 for DSV
Processing Time Interval 3 for 2015-10-26 for ELZ


Processing Time Interval 3 for 2015-11-16 for RME
Processing Time Interval 3 for 2015-11-16 for SCH
Processing Time Interval 3 for 2015-11-16 for SLK
Processing Time Interval 3 for 2015-11-16 for SWF
Processing Time Interval 3 for 2015-11-16 for BUF
Processing Time Interval 3 for 2015-11-16 for JFK
Processing Time Interval 3 for 2015-11-16 for LGA
Processing Time Interval 3 for 2015-11-16 for ROC
Processing Time Interval 3 for 2015-11-16 for SYR
Processing Time Interval 3 for 2015-11-17 for AXB
Processing Time Interval 3 for 2015-11-17 for IUA
Processing Time Interval 3 for 2015-11-17 for FID
Processing Time Interval 3 for 2015-11-17 for HCC
Processing Time Interval 3 for 2015-11-17 for COP
Processing Time Interval 3 for 2015-11-17 for CTO
Processing Time Interval 3 for 2015-11-17 for DKK
Processing Time Interval 3 for 2015-11-17 for DSV
Processing Time Interval 3 for 2015-11-17 for ELZ
Processing Time Interval 3 for 2015-11-17 for FOK
Processing Time Interval 3 for 2015-11-17 for HTO


Processing Time Interval 3 for 2015-12-02 for CTX
Processing Time Interval 3 for 2015-12-02 for SXY
Processing Time Interval 3 for 2015-12-02 for OGS
Processing Time Interval 3 for 2015-12-02 for OIC
Processing Time Interval 3 for 2015-12-02 for ALB
Processing Time Interval 3 for 2015-12-02 for ART
Processing Time Interval 3 for 2015-12-02 for BGM
Processing Time Interval 3 for 2015-12-02 for ELM
Processing Time Interval 3 for 2015-12-02 for FRG
Processing Time Interval 3 for 2015-12-02 for GFL
Processing Time Interval 3 for 2015-12-02 for HPN
Processing Time Interval 3 for 2015-12-02 for IAG
Processing Time Interval 3 for 2015-12-02 for ISP
Processing Time Interval 3 for 2015-12-02 for ITH
Processing Time Interval 3 for 2015-12-02 for JHW
Processing Time Interval 3 for 2015-12-02 for MSS
Processing Time Interval 3 for 2015-12-02 for PBG
Processing Time Interval 3 for 2015-12-02 for POU
Processing Time Interval 3 for 2015-12-02 for RME
Processing Time Interval 3 for 2015-12-02 for SCH


Processing Time Interval 3 for 2015-12-11 for POU
Processing Time Interval 3 for 2015-12-11 for RME
Processing Time Interval 3 for 2015-12-11 for SCH
Processing Time Interval 3 for 2015-12-11 for SLK
Processing Time Interval 3 for 2015-12-11 for SWF
Processing Time Interval 3 for 2015-12-11 for BUF
Processing Time Interval 3 for 2015-12-11 for JFK
Processing Time Interval 3 for 2015-12-11 for LGA
Processing Time Interval 3 for 2015-12-11 for ROC
Processing Time Interval 3 for 2015-12-11 for SYR
Processing Time Interval 3 for 2015-12-12 for AXB
Processing Time Interval 3 for 2015-12-12 for IUA
Processing Time Interval 3 for 2015-12-12 for FID
Processing Time Interval 3 for 2015-12-12 for HCC
Processing Time Interval 3 for 2015-12-12 for COP
Processing Time Interval 3 for 2015-12-12 for CTO
Processing Time Interval 3 for 2015-12-12 for DKK
Processing Time Interval 3 for 2015-12-12 for DSV
Processing Time Interval 3 for 2015-12-12 for ELZ
Processing Time Interval 3 for 2015-12-12 for FOK


Processing Time Interval 3 for 2015-12-26 for DKK
Processing Time Interval 3 for 2015-12-26 for DSV
Processing Time Interval 3 for 2015-12-26 for ELZ
Processing Time Interval 3 for 2015-12-26 for FOK
Processing Time Interval 3 for 2015-12-26 for HTO
Processing Time Interval 3 for 2015-12-26 for WSH
Processing Time Interval 3 for 2015-12-26 for LKP
Processing Time Interval 3 for 2015-12-26 for MGJ
Processing Time Interval 3 for 2015-12-26 for MSV
Processing Time Interval 3 for 2015-12-26 for MTP
Processing Time Interval 3 for 2015-12-26 for CTX
Processing Time Interval 3 for 2015-12-26 for SXY
Processing Time Interval 3 for 2015-12-26 for OGS
Processing Time Interval 3 for 2015-12-26 for OIC
Processing Time Interval 3 for 2015-12-26 for ALB
Processing Time Interval 3 for 2015-12-26 for ART
Processing Time Interval 3 for 2015-12-26 for BGM
Processing Time Interval 3 for 2015-12-26 for ELM
Processing Time Interval 3 for 2015-12-26 for FRG
Processing Time Interval 3 for 2015-12-26 for GFL


### Create DataFrame

In [55]:
 # Creating DataFrame
historical_wx_df = pd.DataFrame({
    'Airport Code': loc_list,
    'Date': date_list,
    'Time': time_list,
    'Latitude': lat,
    'Longitude': lng,
    'Weather Description': weatherDesc,
    'Temperature (F)': tempF,
    'Heat Index (F)': HeatIndexF,
    'Wind Chill (F)': WindChillF,
    'Dew Point (F)': DewPointF,
    'Wind Speed (mph)': windspeedMiles,
    'Wind Gusts (mph)': WindGustMiles,
    'Wind Direction': winddir16Point,
    'Precipitation (in)': precipInches,
    'Humidity (%)': humidity,
    'Visibility (mi)': visibilityMiles,
    'Pressure (mb)': pressure,
    'Cloud Coverage (%)': cloudcover,
    'Snow Total (cm)': totalSnow_cm
})
historical_wx_df

,Airport Code,Date,Time,Latitude,Longitude,Weather Description,Temperature (F),Heat Index (F),Wind Chill (F),Dew Point (F),Wind Speed (mph),Wind Gusts (mph),Wind Direction,Precipitation (in),Humidity (%),Visibility (mi),Pressure (mb),Cloud Coverage (%),Snow Total (cm)
0,AXB,2015-01-01,0,40.780,-73.870,Clear,41,41,35,34,10,15,WNW,0.0,75,6,1025,4,0.0
1,IUA,2015-01-01,0,40.780,-73.870,Clear,41,41,35,34,10,15,WNW,0.0,75,6,1025,4,0.0
2,FID,2015-01-01,0,40.780,-73.870,Clear,41,41,35,34,10,15,WNW,0.0,75,6,1025,4,0.0
3,HCC,2015-01-01,0,40.780,-73.870,Clear,41,41,35,34,10,15,WNW,0.0,75,6,1025,4,0.0
4,COP,2015-01-01,0,40.780,-73.870,Clear,41,41,35,34,10,15,WNW,0.0,75,6,1025,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62775,BUF,2015-12-31,1800,40.780,-73.870,Clear,53,53,51,45,9,18,WSW,0.0,72,6,1022,10,0.0
62776,JFK,2015-12-31,1800,40.780,-73.870,Clear,53,53,51,45,9,18,WSW,0.0,72,6,1022,10,0.0
62777,LGA,2015-12-31,1800,40.780,-73.870,Clear,53,53,51,45,9,18,WSW,0.0,72,6,1022,10,0.0
62778,ROC,2015-12-31,1800,40.780,-73.870,Clear,53,53,51,45,9,18,WSW,0.0,72,6,1022,10,0.0


### Creating CSV File

In [56]:
# Creating CSV File
historical_wx_df.to_csv('../Resources/historical_wx.csv')

### Reading CSV File

In [57]:
 # Reading in CSV File
filepath = '../Resources/historical_wx.csv'
f = pd.read_csv(filepath)
df = pd.DataFrame(f)
df = df.drop(columns=['Unnamed: 0']) # Dropping Column

df.head()

,Airport Code,Date,Time,Latitude,Longitude,Weather Description,Temperature (F),Heat Index (F),Wind Chill (F),Dew Point (F),Wind Speed (mph),Wind Gusts (mph),Wind Direction,Precipitation (in),Humidity (%),Visibility (mi),Pressure (mb),Cloud Coverage (%),Snow Total (cm)
0,AXB,2015-01-01,0,40.78,-73.87,Clear,41,41,35,34,10,15,WNW,0.0,75,6,1025,4,0.0
1,IUA,2015-01-01,0,40.78,-73.87,Clear,41,41,35,34,10,15,WNW,0.0,75,6,1025,4,0.0
2,FID,2015-01-01,0,40.78,-73.87,Clear,41,41,35,34,10,15,WNW,0.0,75,6,1025,4,0.0
3,HCC,2015-01-01,0,40.78,-73.87,Clear,41,41,35,34,10,15,WNW,0.0,75,6,1025,4,0.0
4,COP,2015-01-01,0,40.78,-73.87,Clear,41,41,35,34,10,15,WNW,0.0,75,6,1025,4,0.0
